In [ ]:
#import kagglehub

# Download latest version
#path = kagglehub.dataset_download("kamal01/top-agriculture-crop-disease")

#print("Path to dataset files:", path)

/home/lakshya/jupyter/paper/DFU/Patches/dfu/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.33G/4.33G [14:37<00:00, 5.30MB/s]

Extracting files...


Path to dataset files: /home/lakshya/.cache/kagglehub/datasets/kamal01/top-agriculture-crop-disease/versions/1


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import class_weight

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✅ GPU available and memory growth enabled")
    except RuntimeError as e:
        print(e)
else:
    print("⚠️ No GPU found, training will run on CPU")


✅ GPU available and memory growth enabled


In [4]:
print("TensorFlow version:", tf.__version__)
print("Built with CUDA:", tf.test.is_built_with_cuda())
print("Physical GPUs:", tf.config.list_physical_devices("GPU"))
print("GPU device name:", tf.test.gpu_device_name())


TensorFlow version: 2.19.0
Built with CUDA: True
Physical GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU device name: /device:GPU:0


In [5]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")


In [7]:
import os

# List of directories
DATA_DIRS = [
    "/kaggle/input/top-agriculture-crop-disease/Crop Diseases",
    # "/root/.cache/kagglehub/datasets/laithjj/diabetic-foot-ulcer-dfu/versions/1/DFU/Transfer-Learning images",
]

IMG_SIZE = 224
BATCH = 5
EPOCHS = 30

train_datasets = []
val_datasets = []
test_datasets = []

for data_dir in DATA_DIRS:
    # Load full dataset without validation split
    full_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        image_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH,
        seed=1337,
        shuffle=True,
        label_mode="int"
    )

    # Get class names from the full dataset
    class_names = full_ds.class_names
    print("Classes:", class_names)


    # Get total number of batches
    total_batches = tf.data.experimental.cardinality(full_ds).numpy()

    # Compute split sizes
    train_batches = int(total_batches * 0.7)
    test_batches = int(total_batches * 0.2)
    val_batches = total_batches - train_batches - test_batches  # ~10%

    # Split the dataset
    train_ds = full_ds.take(train_batches)
    test_ds = full_ds.skip(train_batches).take(test_batches)
    val_ds = full_ds.skip(train_batches + test_batches)

    # Append to lists
    train_datasets.append(train_ds)
    val_datasets.append(val_ds)
    test_datasets.append(test_ds)


# Combine datasets
train_ds = train_datasets[0]
for d in train_datasets[1:]:
    train_ds = train_ds.concatenate(d)

val_ds = val_datasets[0]
for d in val_datasets[1:]:
    val_ds = val_ds.concatenate(d)

test_ds = test_datasets[0]
for d in test_datasets[1:]:
    test_ds = test_ds.concatenate(d)

# Optimize pipeline performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.prefetch(buffer_size=AUTOTUNE)

print("✅ Dataset Split Completed:")
print(f"Train batches: {tf.data.experimental.cardinality(train_ds).numpy()}")
print(f"Test batches: {tf.data.experimental.cardinality(test_ds).numpy()}")
print(f"Validation batches: {tf.data.experimental.cardinality(val_ds).numpy()}")

Found 13324 files belonging to 17 classes.
Classes: ['Corn___Common_Rust', 'Corn___Gray_Leaf_Spot', 'Corn___Healthy', 'Corn___Northern_Leaf_Blight', 'Potato___Early_Blight', 'Potato___Healthy', 'Potato___Late_Blight', 'Rice___Brown_Spot', 'Rice___Healthy', 'Rice___Leaf_Blast', 'Rice___Neck_Blast', 'Sugarcane_Bacterial Blight', 'Sugarcane_Healthy', 'Sugarcane_Red Rot', 'Wheat___Brown_Rust', 'Wheat___Healthy', 'Wheat___Yellow_Rust']
✅ Dataset Split Completed:
Train batches: 1865
Test batches: 533
Validation batches: 267


In [8]:
labels = np.concatenate([y.numpy() for _, y in train_ds], axis=0)
cw_vals = class_weight.compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights = {int(c): float(w) for c, w in zip(np.unique(labels), cw_vals)}
print("Class weights:", class_weights)

Class weights: {0: 0.6917142645204362, 1: 1.4825119236883944, 2: 0.6822505121451565, 3: 0.7903882013900662, 4: 0.7758548964140112, 5: 5.540701128936423, 6: 0.7984416474013186, 7: 1.2108817036748474, 8: 0.5341084827309697, 9: 0.793819698646463, 10: 0.797281121751026, 11: 7.313725490196078, 12: 8.066608996539792, 13: 6.530112044817927, 14: 0.8570772058823529, 15: 0.6822505121451565, 16: 0.8557401119574195}


In [9]:
inputs = layers.Input((IMG_SIZE, IMG_SIZE, 3))
x = layers.Rescaling(1./127.5, offset=-1)(inputs)

base = tf.keras.applications.MobileNetV2(include_top=False,
                                         input_shape=(IMG_SIZE,IMG_SIZE,3),
                                         weights="imagenet")
base.trainable = False   # freeze base for initial training

x = base(x, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(len(class_names), activation="softmax", dtype="float32")(x)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [10]:
model = models.Model(inputs, outputs)
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 1280)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       327,936 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 17)             │         4,369 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,590,289 (9.88 MB)

 Trainable params: 332,305 (1.27 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [11]:
ckpt = callbacks.ModelCheckpoint("best_mobilenetv2_gpu.h5", save_best_only=True,
                                 monitor="val_accuracy", mode="max")
es = callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)


In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[ckpt, es],
    class_weight=class_weights
)


Epoch 1/30
1860/1865 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7156 - loss: 1.0915


1865/1865 ━━━━━━━━━━━━━━━━━━━━ 251s 101ms/step - accuracy: 0.7158 - loss: 1.0904 - val_accuracy: 0.8838 - val_loss: 0.2934
Epoch 2/30
1865/1865 ━━━━━━━━━━━━━━━━━━━━ 223s 99ms/step - accuracy: 0.8532 - loss: 0.4434 - val_accuracy: 0.8823 - val_loss: 0.2976
Epoch 3/30
1863/1865 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - accuracy: 0.8760 - loss: 0.3355


1865/1865 ━━━━━━━━━━━━━━━━━━━━ 186s 77ms/step - accuracy: 0.8760 - loss: 0.3356 - val_accuracy: 0.9138 - val_loss: 0.2228
Epoch 4/30
1865/1865 ━━━━━━━━━━━━━━━━━━━━ 224s 100ms/step - accuracy: 0.8777 - loss: 0.3519 - val_accuracy: 0.8913 - val_loss: 0.2561
Epoch 5/30
1865/1865 ━━━━━━━━━━━━━━━━━━━━ 223s 100ms/step - accuracy: 0.8939 - loss: 0.2569 - val_accuracy: 0.9093 - val_loss: 0.2600
Epoch 6/30
1865/1865 ━━━━━━━━━━━━━━━━━━━━ 178s 75ms/step - accuracy: 0.8886 - loss: 0.3228 - val_accuracy: 0.9123 - val_loss: 0.2365
Epoch 7/30
1860/1865 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.9007 - loss: 0.3013

In [ ]:
!nvidia-smi

In [ ]:
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    callbacks=[ckpt, es],
    class_weight=class_weights
)


In [ ]:
loss, acc = model.evaluate(val_ds)
print(f"✅ Validation Loss: {loss:.16f}")
print(f"✅ Validation Accuracy: {acc:.16f}")

In [ ]:
import os

# Search for .h5 or SavedModel folders recursively under your project
for root, dirs, files in os.walk(".", topdown=True):
    for f in files:
        if f.endswith(".h5"):
            print("Found model file:", os.path.join(root, f))
    if "saved_model.pb" in files:
        print("Found TensorFlow SavedModel:", root)


In [ ]:
import os
import sys
import csv
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
TEST_DIR = "/root/.cache/kagglehub/datasets/laithjj/diabetic-foot-ulcer-dfu/versions/1/DFU/TestSet"        # change to your test folder if different
MODEL_PATH = "./best_mobilenetv2_gpu.h5"  # found model file
IMG_SIZE = 224
BATCH = 32
OUT_CSV = "test_predictions.csv"
CONF_MATRIX_PNG = "confusion_matrix.png"
LABELS_CSV_NAME = "labels.csv"

In [ ]:
IMG_EXTS = ('.png', '.jpg', '.jpeg', '.bmp', '.tif', '.tiff')

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for g in gpus:
            tf.config.experimental.set_memory_growth(g, True)
        print("✅ GPU(s) found - memory growth enabled")
    except Exception as e:
        print("⚠️ GPU setup issue:", e)
else:
    print("⚠️ No GPU found - running on CPU")


In [ ]:
def discover_images(root):
    paths = []
    for r, _, files in os.walk(root):
        for f in files:
            if f.lower().endswith(IMG_EXTS):
                paths.append(os.path.join(r, f))
    return sorted(paths)


In [ ]:
def auto_infer_label_from_filename(fname):
    lower = fname.lower()
    KEYWORD_LABELS = {
        "infect": "infected",
        "ulcer": "infected",
        "pus": "infected",
        "abscess": "infected",
        "healthy": "healthy",
        "normal": "healthy",
        "noninfect": "healthy",
        "noinfect": "healthy"
    }
    for k, v in KEYWORD_LABELS.items():
        if k in lower:
            return v
    return "unknown"


In [ ]:
def load_batch_images(paths, target_size):
    arr = np.zeros((len(paths), target_size, target_size, 3), dtype=np.float32)
    for i, p in enumerate(paths):
        img = image.load_img(p, target_size=(target_size, target_size))
        a = image.img_to_array(img).astype(np.float32)
        a = (a / 127.5) - 1.0  # MobileNetV2 expected preprocessing
        arr[i] = a
    return arr


In [ ]:
if not os.path.isdir(TEST_DIR):
    print(f"ERROR: TEST_DIR does not exist: {TEST_DIR}")
    sys.exit(1)

all_images = discover_images(TEST_DIR)
print(f"Discovered {len(all_images)} image files under {TEST_DIR}")
if len(all_images) == 0:
    print("No image files found. Check TEST_DIR and extensions.")
    sys.exit(1)

In [ ]:
parents = [os.path.basename(os.path.dirname(p)) for p in all_images]
unique_parents = sorted(set(parents))

use_subfolders = False
if len(unique_parents) > 1:
    test_dir_base = os.path.basename(os.path.normpath(TEST_DIR))
    if any(p != test_dir_base for p in unique_parents):
        use_subfolders = True

filepaths = []
y_true = []
class_names = None


In [ ]:
if use_subfolders:
    print("Detected class subfolders; inferring labels from immediate parent folder names.")
    class_names = sorted(list(set(parents)))
    class_to_idx = {c: i for i, c in enumerate(class_names)}
    for p in all_images:
        cls = os.path.basename(os.path.dirname(p))
        filepaths.append(p)
        y_true.append(class_to_idx[cls])
    print("Classes:", class_names)
else:
    print("Images appear to be in a single folder. Looking for labels CSV...")
    csv_candidates = [os.path.join(TEST_DIR, LABELS_CSV_NAME),
                      os.path.join(TEST_DIR, "test_labels.csv"),
                      os.path.join(TEST_DIR, "labels.csv")]
    csv_path = None
    for c in csv_candidates:
        if os.path.isfile(c):
            csv_path = c
            break

    if csv_path:
        print("Found labels CSV:", csv_path)
        df = pd.read_csv(csv_path)
        cols_lower = [c.lower() for c in df.columns]
        if 'filename' in cols_lower and 'label' in cols_lower:
            fname_col = df.columns[cols_lower.index('filename')]
            label_col = df.columns[cols_lower.index('label')]
        else:
            fname_col = df.columns[0]
            label_col = df.columns[1]
        mapping = {}
        for _, row in df.iterrows():
            base = os.path.basename(str(row[fname_col]))
            mapping[base] = str(row[label_col])
        matched = 0
        for p in all_images:
            b = os.path.basename(p)
            if b in mapping:
                filepaths.append(p)
                y_true.append(mapping[b])
                matched += 1
        if matched == 0:
            print("ERROR: labels CSV found but filenames did not match discovered images.")
            sys.exit(1)
        unique_labels = sorted(list(set(y_true)))
        class_names = unique_labels
        class_to_idx = {c: i for i, c in enumerate(class_names)}
        y_true = [class_to_idx[s] for s in y_true]
        print(f"Loaded {len(filepaths)} labeled images from CSV. Classes: {class_names}")
    else:
        print("No labels CSV found. Attempting to auto-infer labels from filenames (keywords).")
        rows = []
        for p in all_images:
            base = os.path.basename(p)
            inferred = auto_infer_label_from_filename(base)
            rows.append((base, inferred))
        out_csv_path = os.path.join(TEST_DIR, LABELS_CSV_NAME)
        with open(out_csv_path, "w", newline="") as fh:
            writer = csv.writer(fh)
            writer.writerow(["filename", "label"])
            writer.writerows(rows)
        print(f"Wrote inferred labels to {out_csv_path}. Please edit if many are 'unknown'.")

        good = [(f, lbl) for f, lbl in rows if lbl != "unknown"]
        if len(good) == 0:
            print("Auto-inference produced 0 labeled images (all 'unknown'). Open labels.csv and fill labels.")
            sys.exit(1)
        print(f"Auto-inferred {len(good)} labeled images; proceeding with those (others are 'unknown').")
        class_names = sorted(list(set(lbl for _, lbl in good)))
        class_to_idx = {c: i for i, c in enumerate(class_names)}
        for fname, lbl in good:
            filepaths.append(os.path.join(TEST_DIR, fname))
            y_true.append(class_to_idx[lbl])

if len(filepaths) == 0:
    print("No labeled images prepared for evaluation.")
    sys.exit(1)

filepaths = np.array(filepaths)
y_true = np.array(y_true, dtype=int)
print(f"Prepared {len(filepaths)} labeled images. Classes: {class_names}")


In [ ]:
if not os.path.isfile(MODEL_PATH):
    print(f"ERROR: Model file not found: {MODEL_PATH}")
    sys.exit(1)

print("Loading model:", MODEL_PATH)
model = tf.keras.models.load_model(MODEL_PATH)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
print("Model loaded and compiled.")


In [ ]:
n = len(filepaths)
pred_logits_list = []
for i in range(0, n, BATCH):
    batch_paths = filepaths[i:i+BATCH]
    batch_imgs = load_batch_images(batch_paths, IMG_SIZE)
    logits = model.predict(batch_imgs, verbose=0)
    pred_logits_list.append(logits)
pred_logits = np.vstack(pred_logits_list)
pred_idxs = np.argmax(pred_logits, axis=1)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
if 'y_true' not in globals() or 'pred_idxs' not in globals():
    raise RuntimeError("y_true or pred_idxs missing — run prediction block first.")


In [ ]:
unique_true = np.unique(y_true)
num_classes = len(unique_true)
print("Unique label indices in y_true:", unique_true, "num_classes:", num_classes)


In [ ]:
if 'class_to_idx' in globals() and isinstance(class_to_idx, dict):
    inv_map = {v: k for k, v in class_to_idx.items()}
    # create an indexed list where index i maps to label name if present, else fallback
    repaired_names = []
    for i in range(num_classes):
        repaired_names.append(inv_map.get(i, f"class_{i}"))
    class_names = repaired_names
else:
    # If class_names exists and is a list, try to pad/truncate it
    if 'class_names' in globals() and isinstance(class_names, (list, tuple)):
        class_names = list(class_names)
        if len(class_names) < num_classes:
            # pad with generic names
            class_names += [f"class_{i}" for i in range(len(class_names), num_classes)]
        elif len(class_names) > num_classes:
            # truncate (assume extra names were spurious)
            class_names = class_names[:num_classes]
    else:
        # fallback default names
        class_names = [f"class_{i}" for i in range(num_classes)]

print("Using class_names (index -> name):")
for i, nm in enumerate(class_names):
    print(f"  {i} -> {nm}")


In [ ]:
all_label_indices = sorted(set(np.unique(np.concatenate([y_true, pred_idxs]))))
if max(all_label_indices) >= num_classes:
    # If predicted label indices exceed num_classes, remap predicted labels to contiguous range if possible
    print("Warning: predicted labels contain indices outside the ground-truth label set.")
    # We'll still pass explicit labels to sklearn to avoid mismatch; expand class_names if needed
    max_label = max(all_label_indices)
    if len(class_names) <= max_label:
        class_names += [f"class_{i}" for i in range(len(class_names), max_label+1)]
        num_classes = len(class_names)
        print("Expanded class_names to cover predicted labels. New num_classes:", num_classes)


In [ ]:
labels_for_sklearn = list(range(num_classes))

In [ ]:
report = classification_report(y_true, pred_idxs, labels=labels_for_sklearn, target_names=class_names[:len(labels_for_sklearn)], digits=4)
print("\nClassification Report:\n", report)


In [ ]:
cm = confusion_matrix(y_true, pred_idxs, labels=labels_for_sklearn)
print("Confusion matrix:\n", cm)


In [ ]:
accuracy = (pred_idxs == y_true).mean()
print(f"Computed Accuracy: {accuracy:.4f}")


In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 6))
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.figure.colorbar(im, ax=ax)
ax.set(xticks=np.arange(len(class_names[:len(labels_for_sklearn)])),
       yticks=np.arange(len(class_names[:len(labels_for_sklearn)])),
       xticklabels=class_names[:len(labels_for_sklearn)],
       yticklabels=class_names[:len(labels_for_sklearn)],
       ylabel='True label', xlabel='Predicted label', title='Confusion Matrix')
plt.setp(ax.get_xticklabels(), rotation=45, ha="right")
thresh = cm.max() / 2. if cm.max() > 0 else 0
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > thresh else "black")
plt.tight_layout()
plt.savefig(CONF_MATRIX_PNG, dpi=150)
plt.show()
print(f"Saved confusion matrix to {CONF_MATRIX_PNG}")


In [ ]:
import pandas as pd
pred_prob = pred_logits.max(axis=1) if 'pred_logits' in globals() else np.max(pred_idxs, axis=1)
out_df = pd.DataFrame({
    "filepath": filepaths,
    "true_idx": y_true,
    "true_label": [class_names[i] for i in y_true],
    "pred_idx": pred_idxs,
    "pred_label": [class_names[i] for i in pred_idxs],
    "pred_prob": pred_prob
})
out_df.to_csv(OUT_CSV, index=False)
print(f"Saved predictions to {OUT_CSV}")
